# RNN PLAY GENERATOR

In [2]:
%tensorflow_version 2.x
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


***Dataset***

In [3]:
path_to_file = tf.keras.utils.get_file('shakespare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1130496/1115394 [==============================] - 0s 0us/step


***Loading your own data***

In [4]:
from google.colab import files
path_to_file = list(files.upload().keys())[0]

IndexError: ignored

# Read contents of file
Let's look at the contents of file

In [5]:
# Read and then decode for py2 compat
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

# length of the text is the number of characters in it
print('length of text: {} characters'.format(len(text)))

length of text: 1115394 characters


In [6]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



# Encoding
We will encode the each unique character

In [7]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
def text_to_int(text): 
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)


In [8]:
#lets look at how part of our text is encoded
print("Text:", text[:13])
print("Encoded", text_to_int(text[:13]))

Text: First Citizen
Encoded [18 47 56 57 58  1 15 47 58 47 64 43 52]


Here we will make a function which converts numeric to text

In [9]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return '' .join(idx2char[ints])
print(int_to_text(text_as_int[:13]))

First Citizen


# Creating Training Examples

input: Hell | output: ello

In [10]:
seq_length = 100 # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

Next we can use the batch method to turn this stream of characters into batches of desired length

In [11]:
sequences = char_dataset.batch(seq_length+1, drop_remainder = True)

Now we need to use these sequences of length 101 and split them into input and output

In [12]:
def split_input_target(chunk):
  input_text = chunk[:-1] #hell
  target_text = chunk[1:] #ello
  return input_text, target_text #hell, ello

dataset = sequences.map(split_input_target) # we use map to apply the above function to every entery

In [13]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [14]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab) # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).


BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# Building the Model
Now it is time to build the model.We will use an embedding layer a LSTM and one dense layer that contains a node for each unique character
in our training data.The dense layer will give us a probability distribution over all nodes.

In [15]:
def build_model(vocab_size, embedding_dim, rnn_uints, batch_size):
  model = tf.keras.Sequential([
      tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                batch_input_shape = [batch_size, None]),
      tf.keras.layers.LSTM(rnn_uints,
                           return_sequences = True,
                           stateful = True,
                           recurrent_initializer="glorot_uniform"),
      tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


# Creating a Loss Function
Now we are actually going to create our own loss function for this problem.This is because our model will output a (64,sequence_length,65)
shaped tensor that represents the probability distribution of each character at each timestep for every sequence in the batch.
However,before we do that let's have a look at a sample input and the output from our untrained model. This is so we can understand what the
model is actually giving us.

In [ ]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch) # ask our model for aprediction on our first batch of training dataset
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)") # print out the output shape

In [18]:
# We can see that the prdicition id an array of 64 arrays, one for each entry in batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[ 3.34185339e-03 -2.02148920e-03 -3.10661737e-04 ... -4.31288080e-03
   -7.68235233e-03 -7.26199569e-03]
  [-1.08446670e-03 -5.53868758e-03 -5.54007944e-03 ... -8.31630453e-03
   -6.13982044e-03 -3.93472798e-03]
  [-2.06763670e-03 -8.17733537e-03 -3.53134773e-03 ... -3.69916158e-03
   -5.34433825e-03  3.54399579e-03]
  ...
  [ 7.75567302e-03 -8.40554014e-04  4.37017111e-03 ...  4.85200481e-03
   -1.81558030e-03 -6.78229751e-03]
  [ 6.54204283e-03 -1.86844170e-03  1.02391671e-02 ...  2.31172051e-03
   -5.00449957e-03 -1.09308343e-02]
  [ 1.30025297e-03 -7.00790342e-03  2.45270412e-03 ... -4.10414580e-03
   -4.76972573e-03 -5.86616574e-03]]

 [[ 5.42269927e-03 -3.03143589e-03  1.14854169e-03 ...  8.26217979e-03
   -5.01004793e-03 -4.94214473e-03]
  [ 9.17511433e-03  4.69616335e-03  2.81470455e-03 ...  7.63871148e-03
   -4.56173113e-03 -6.84557110e-03]
  [ 1.87530788e-03 -1.32079143e-03 -4.66363272e-03 ...  9.98481177e-04
   -5.68996836e-03 -4.34946269e-03]
  ...
  [ 4.484

In [19]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# norice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[ 0.00334185 -0.00202149 -0.00031066 ... -0.00431288 -0.00768235
  -0.007262  ]
 [-0.00108447 -0.00553869 -0.00554008 ... -0.0083163  -0.00613982
  -0.00393473]
 [-0.00206764 -0.00817734 -0.00353135 ... -0.00369916 -0.00534434
   0.003544  ]
 ...
 [ 0.00775567 -0.00084055  0.00437017 ...  0.004852   -0.00181558
  -0.0067823 ]
 [ 0.00654204 -0.00186844  0.01023917 ...  0.00231172 -0.0050045
  -0.01093083]
 [ 0.00130025 -0.0070079   0.0024527  ... -0.00410415 -0.00476973
  -0.00586617]], shape=(100, 65), dtype=float32)


In [20]:
# and finally we will look at prediction at hte first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probability of each character occuring next

65
tf.Tensor(
[ 3.3418534e-03 -2.0214892e-03 -3.1066174e-04  7.2875815e-03
 -5.1059709e-03  7.1285553e-03  4.2333649e-03  1.1268966e-03
 -8.5847927e-03 -1.0133115e-02 -4.8522735e-03 -3.4773601e-03
 -9.2468364e-03 -1.1025535e-02  9.2735235e-03  2.6395719e-03
  6.8964418e-03  9.5092263e-03 -5.6459457e-03 -2.0740964e-03
 -9.5017962e-03  1.1426643e-03  1.0840213e-02 -4.6393443e-03
 -3.7423589e-03  1.3944695e-02 -6.3461517e-03  1.8132052e-03
  3.2960984e-04 -4.9541215e-03 -6.9212983e-03 -7.2124312e-03
  1.7482671e-03  1.6848044e-04  6.6290647e-03  3.7664408e-03
  1.5919362e-03 -6.6288514e-05 -1.3196230e-02 -4.6210438e-03
 -1.5082979e-04 -1.2378690e-02 -1.4974058e-03 -2.9404084e-03
  4.2894180e-03  5.1829764e-03 -9.2938272e-03 -1.6291628e-02
  1.0023757e-02 -3.9267065e-03 -6.2557557e-03 -5.8042491e-04
 -1.9532986e-02  8.5294815e-03 -5.2936524e-03 -4.4688843e-03
  1.5542865e-03 -7.1273400e-03 -9.4855335e-03 -3.5324655e-03
 -7.5431401e-03  3.6858378e-03 -4.3128808e-03 -7.6823523e-03
 -7.261995

In [21]:
# If we want to determine the predicted character we need to sample the output distribution
sampled_indices = tf.random.categorical(pred, num_samples=1)

# #now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)
predicted_chars # and this is what the model predicted for training sequence 1

"Pdb&kG&O.w VQSbvpAKeu\n?,K!Nna:VS-NVEYPCGv-FoqrR\nRka\nzjO3RcghrannQ.:it 'R!!QhLW!IKgTMITWGH3c-CAxk;IXM"

So now we need to create a loss function that can compare that output to the expected output and give us some numeric value representing
how close the two were.

In [22]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

# Compiling the Model
At this point we can think of our problem as a classification problem where the model predicts the probabillity of each unique letter coming
next.

In [23]:
model.compile(optimizer = 'adam', loss = loss)

# Creating Checkpoints
Now we are going to setup and configure our model to save checkpoinst as it trains.This will allow us to load our model from a checkpoint
and continue training it.

In [24]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_prefix,
    save_weights_only=True)

# Training
Finally we will start training the model.

In [25]:
history = model.fit(data, epochs=2, callbacks = [checkpoint_callback])

Epoch 1/2
172/172 [==============================] - 1272s 7s/step - loss: 2.5625
Epoch 2/2
172/172 [==============================] - 1171s 7s/step - loss: 1.8616


# Loading the Model
We'll rebuild the model from a checkpoint using a batch_size of 1 so that we can feed one peice of text to the model and have it make a
prediction.

In [26]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

Once the model is finished training we can find the lastest checkpoint that stores the models weights using the following line.

In [27]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

We can load any checkpoint we want by specifying the exact file to load.

In [30]:
checkpoint_num = 10
model.load_weights(tf.train.load_checkpoint("./ training checkpoints/ckpt_" + str(checkpoint_num)))
model.build(tf.TensorShape([1, None]))

NotFoundError: ignored

# Generating Text
Now we can use the lovely function provided by tensorflow to generate some text using any starting string we'd like.

In [32]:
def generate_text(model, start_string):
# Evaluation step (generating text using the learned model)

#Number of characters to generate
  num_generate = 800
# Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)
# Empty string to store our results
  text_generated = []
# Low temperatures results in more predictable text.
# Higher temperatures results in more surprising text.
# Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    # remove the batch dimension
    predictions = tf.squeeze(predictions, 0)
    # using a categorical distribution to predict the character returned by the model
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
    # We pass the predicted character as the next input to the model
    # along with the previous hidden state
    input_eval = tf.expand_dims([predicted_id], 0)
    text_generated.append(idx2char[predicted_id])
  return (start_string + ' '.join(text_generated))
    

In [33]:
inp = input("Type a starting string: ")
print(generate_text(model, inp)) 

Type a starting string: romeo
romeou ? 
 
 P O M I L E N E S : 
 M e t a y ,   a n d   I   s a y ,   w h o m   I   h a v e ,   m a d e ,   s h o   g u t s e l f   b e ,   s i r ? 
 W h o ,   w i l l   I   w o u l d   o u t h r a r d   t h e   k l o w   d e a r t h o n d 
 S e a l l y   b u t   d i d l e d   y o u r   p r i n c u s e   a s   y o u   m a k e   t o o d 
 B e t h i c e d e n   e x t   w h o s   t h a t   h i m   o r e   i t   p a r d   c o u c h , 
 S e c a r e   f o r r o w i o n ' s   w i t h   t h e s   t h r e a t h e r ,   h a v e   n o t   t h e   s e a v e r . 
 N o r t o m e   y o u   s h o w n d   i n e   s h e l i v e d ,   s h a l l   c o m e ,   h e   s w a i t h y   a l e . 
 
 F i r s t   F o r n r e r : 
 ' T i s   l o r d   l a c a i n e d ' s   r i e   a m y   a n d   t h a t   p r o c e r . 
 
 A S T I A N S : 
 A t h e e   r i s t i s e   a   m o r e   b e e n   a n d   s h a l l 
 T o u c k ' d   a   k n o u g h t h . 
 
 R O M T R S A M E R L A : 
 S o